# Spam Classification Using MLP

**IMPORTS REQUIRED FOR THE MODEL**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

**LOADING DATASET**

In [ ]:
file_path = "/content/spam.csv"
df = pd.read_csv(file_path, encoding="latin-1")
df.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


**Selecting Relevant Columns**

In [ ]:
df = df[['Category', 'Message']]

**Removing duplicates**

In [ ]:
df = df.drop_duplicates().dropna()
#This prevents the model from learning redundant information and ensures the dataset has no empty values that could cause errors.

**Label Encoding**

In [ ]:
# Encode labels (spam = 1, ham = 0)
label_encoder = LabelEncoder()
df["Category"] = label_encoder.fit_transform(df["Category"])
#Models need numbers, not text. Encoding ensures consistency and prevents errors in training

**Text Vectorization using TF-IDF**

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X = vectorizer.fit_transform(df["Message"]).toarray()
y = df["Category"].values.reshape(-1, 1)
#Models need numbers, not text. Encoding ensures consistency and prevents errors in training

**Split Dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**MLP NEURAL NETWORK**


In [ ]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
        self.b2 = np.zeros((1, self.output_size))
    # Sigmoid activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Derivative of sigmoid (used in backpropagation)
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Forward propagation
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)  # Using Sigmoid instead of ReLU
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    # Backpropagation (weight update)
    def backward(self, X, y):
        m = y.shape[0]

        # Compute error
        error = self.a2 - y
        d_output = error * self.sigmoid_derivative(self.a2)

        # Compute gradient for hidden layer
        error_hidden = np.dot(d_output, self.W2.T)
        d_hidden = error_hidden * self.sigmoid_derivative(self.a1)

        # Update weights and biases
        self.W2 -= self.learning_rate * np.dot(self.a1.T, d_output) / m
        self.b2 -= self.learning_rate * np.sum(d_output, axis=0, keepdims=True) / m
        self.W1 -= self.learning_rate * np.dot(X.T, d_hidden) / m
        self.b1 -= self.learning_rate * np.sum(d_hidden, axis=0, keepdims=True) / m

    # Train the model
    def train(self, X, y, epochs=500):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)

            if epoch % 100 == 0:
                loss = np.mean((self.a2 - y) ** 2)
                print(f"Epoch {epoch}, Loss: {loss:.6f}")

    # Predict function
    def predict(self, X):
        predictions = self.forward(X)
        return (predictions > 0.5).astype(int)

# Train the MLP model
mlp = MLP(input_size=X_train.shape[1], hidden_size=256, output_size=1, learning_rate=0.01)
mlp.train(X_train, y_train, epochs=500)

# Make predictions
predictions = mlp.predict(X_test)

# Evaluate accuracy
accuracy = np.mean(y_pred == y_test)*100
print(f"Final Accuracy: {accuracy:.2f}%")



Epoch 0, Loss: 0.259868
Epoch 100, Loss: 0.109332
Epoch 200, Loss: 0.107667
Epoch 300, Loss: 0.107476
Epoch 400, Loss: 0.107443
Final Accuracy: 86.82%
